In [ ]:
#Import Packages
import numpy as np
import pandas as pd
import bioinfokit
from bioinfokit import visuz as bfvz
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Read in reports generated in bash_script.sh, reformat into .tsv and .bed
name_list = ['H3K27me3_fruDelvCntrl', 'H3K27ac_fruDelvCntrl', 'H3K4me3_fruDelvCntrl']
!mkdir formatted_beds
!mkdir final
for name in name_list:
    df = pd.read_csv("Reports/" + name, sep = '\t', skiprows = 32)    

    df['Identifier'] = np.arange(len(df))
    df['Chrom'] ='chr' + df['Chrom'].astype(str)
    
    bed = ['Chrom', 'Start', 'End', 'Identifier']
    df[bed].to_csv("formatted_beds/" + name + ".bed", sep='\t', index=False, header=False)

In [ ]:
#Format and read annotate which peaks are bound by fruC based on fruC-myc-PEAKS.bed (see bash_scripts.sh)
for name in name_list:
    df_anno = pd.read_csv("annotations/" + name, sep = '\t')
    df_anno = df_anno.sort_values(by=df_anno.columns[0])
    df_anno = df_anno.reset_index()
    
    main_headers = ['Identifier', 'Chrom', 'Start', 'End', 'Event', 'Length', 'log2FC', 'padj']
    anno = ['Gene Name', 'Annotation']
    df_final = df[main_headers]
    df_final = df_final.join(df_anno[anno])
    df_final.to_csv("final/" + name + ".tsv", sep='\t', index=False, header=True)

In [ ]:
#Format and read annotate which peaks are bound by fruC based on fruC-myc-PEAKS.bed (see bash_scripts.sh)
name_list = ['H3K27me3_fruDelvCntrl', 'H3K27ac_fruDelvCntrl', 'H3K4me3_fruDelvCntrl']
for name in name_list:
    df_final = pd.read_csv("final/" + name + ".tsv", sep = '\t')
    df_fru_bound = pd.read_csv("fru_bound/" + name, sep = '\t', header=None)

    df_final['fru_bound'] = False
    i=0
    for x in df_final['fru_bound']:
        if i in df_fru_bound[3].tolist():
            df_final.at[i,'fru_bound'] = True
        i=i+1
    df_final['fru_bound']
    df_fru_bound_final = df_final[df_final['fru_bound']==True]
    
    #generate volcano plots in Fig5.C-D, FigS5.C,D,F,G
    bfvz.GeneExpression.volcano(df_final, 'log2FC', 'padj', lfc_thr=1, pv_thr=0.05, xlm = (-5,5,1), dim = (10,10), gfont = 15,  
                                sign_line = True)
    path = "figures/" + name + "all.png"
    !mv volcano.png {path}
    bfvz.GeneExpression.volcano(df_fru_bound_final, 'log2FC', 'padj', lfc_thr=1, pv_thr=0.05, xlm = (-5,5,1), dim = (10,10), gfont = 15,  
                                sign_line = True)
    path = "figures/" + name + "nb.png"
    !mv volcano.png {path}

    #--------------------------------------
    #Save dataframes to new variables, and output number of significant events
    #--------------------------------------
    if(name=='H3K4me3_fruDelvCntrl'):
        df_final_k4=df_final
        df_final_k4_fru=df_fru_bound_final
        
    if(name=='H3K27me3_fruDelvCntrl'):
        df_final_k27me3=df_final
        df_final_k27me3_fru=df_fru_bound_final
    
    if(name=='H3K27ac_fruDelvCntrl'):
        df_final_k27ac=df_final
        df_final_k27ac_fru=df_fru_bound_final
        
    print(name)
    print("All Regions:")
    print("Upregulated Events:", sum((df_final['Event']=='Up') & (df_final['log2FC']>1) & (df_final['padj']<0.05)))
    print("Base Pairs:", \
          sum(df_final['End'][(df_final['Event']=='Up') & (df_final['log2FC']>1) & (df_final['padj']<0.05)]\
             -df_final['Start'][(df_final['Event']=='Up') & (df_final['log2FC']>1) & (df_final['padj']<0.05)]))
    
    print("Downregulated Events:", sum((df_final['Event']=='Down') & (df_final['log2FC']<-1) & (df_final['padj']<0.05)))
    print("Base Pairs:", \
      sum(df_final['End'][(df_final['Event']=='Down') & (df_final['log2FC']<-1) & (df_final['padj']<0.05)]\
         -df_final['Start'][(df_final['Event']=='Down') & (df_final['log2FC']<-1) & (df_final['padj']<0.05)]))
   
    print("Total Events:", len(df_final.index))
    print("Base Pairs:", sum(df_final['End']-df_final['Start']))
    
    
    print("Fru Bound:")
    
    print("Upregulated Events:", sum((df_fru_bound_final['Event']=='Up') & (df_fru_bound_final['log2FC']>1) & (df_fru_bound_final['padj']<0.05)))
    print("Base Pairs:", \
          sum(df_fru_bound_final['End'][(df_fru_bound_final['Event']=='Up') & (df_fru_bound_final['log2FC']>1) & (df_fru_bound_final['padj']<0.05)]\
             -df_fru_bound_final['Start'][(df_fru_bound_final['Event']=='Up') & (df_fru_bound_final['log2FC']>1) & (df_fru_bound_final['padj']<0.05)]))
    
    print("Downregulated Events:", sum((df_fru_bound_final['Event']=='Down') & (df_fru_bound_final['log2FC']<-1) & (df_fru_bound_final['padj']<0.05)))
    print("Base Pairs:", \
      sum(df_fru_bound_final['End'][(df_fru_bound_final['Event']=='Down') & (df_fru_bound_final['log2FC']<-1) & (df_fru_bound_final['padj']<0.05)]\
         -df_fru_bound_final['Start'][(df_fru_bound_final['Event']=='Down') & (df_fru_bound_final['log2FC']<-1) & (df_fru_bound_final['padj']<0.05)]))
   
    print("Total Events:", len(df_fru_bound_final.index))
    print("Base Pairs:", sum(df_fru_bound_final['End']-df_fru_bound_final['Start']))

In [ ]:
#Read in histogram counts generated in bash_script.sh
col_names = ["chr", "start", "end", "bin_name", "K4me3_1", "K4me3_2", "K4me3_df_1", "K4me3_df_2", "K27ac_1", "K27ac_2", "K27ac_df_1", "K27ac_df_2", "K27me3_1", "K27me3_2", "K27me3_df_1", "K27me3_df_2"]
df_hists = pd.read_csv("../hists/multi_cov_subsample_fru_unbound.txt", sep = '\t', header=None, names=col_names)
df_hists_fru = pd.read_csv("../hists/multi_cov_subsample_fru_bound.txt", sep = '\t', header=None, names=col_names)

In [ ]:
#Take average of replicate values 
df_hists["K27me3"] = df_hists[['K27me3_1', 'K27me3_2']].mean(axis=1)
df_hists["K27me3_df"] = df_hists[['K27me3_df_1', 'K27me3_df_2']].mean(axis=1)

df_hists_fru["K27me3"] = df_hists_fru[['K27me3_1', 'K27me3_2']].mean(axis=1)
df_hists_fru["K27me3_df"] = df_hists_fru[['K27me3_df_1', 'K27me3_df_2']].mean(axis=1)

In [ ]:
#Generate Density Plot, Fig 5F (Left)
matplotlib.rcParams['axes.linewidth'] = 3 
matplotlib.rcParams['axes.autolimit_mode'] = 'round_numbers'
matplotlib.rcParams['axes.xmargin'] = 0
matplotlib.rcParams['axes.ymargin'] = 0

marks = ["K27me3", "K27me3_df"]
for mark in marks:
    subset = df_hists[mark]
    subset[subset>150] = 150 
    sns.distplot(subset, hist = False, kde = True,
                 kde_kws = {'linewidth': 4},
                 label = mark)
    
# Plot formatting
plt.xlim([0,150])
plt.ylim([0,0.04])

ax = plt.gca()

ax.axes.xaxis.set_ticklabels([])
ax.axes.yaxis.set_ticklabels([])

ax.xaxis.set_tick_params(length = 20, width=3)
ax.yaxis.set_tick_params(length = 20, width=3)
plt.xticks([50, 100, 150])
plt.yticks(np.linspace(plt.yticks()[0][0], plt.yticks()[0][-1],5))


plt.title('')
plt.xlabel('')
plt.ylabel('')
plt.savefig('figures/5F_Left.png', dpi=450)

In [ ]:
#Generate Density Plot, Fig 5F (Right)
marks = ["K27me3", "K27me3_df"]
df_sub = df_hists_fru[marks]
subset = df_sub[marks]
    
for mark in marks:
    subset = df_sub[mark]
    subset[subset>150] = 150
    sns.distplot(subset, hist = False, kde = True,
                 kde_kws = {'linewidth': 4},
                 label = mark)
    
# Plot formatting
plt.xlim([0,150])
plt.ylim([0,0.06])

ax = plt.gca()

ax.axes.xaxis.set_ticklabels([])
ax.axes.yaxis.set_ticklabels([])

ax.xaxis.set_tick_params(length = 20, width=3)
ax.yaxis.set_tick_params(length = 20, width=3)
plt.xticks([50, 100, 150])
plt.yticks(np.linspace(plt.yticks()[0][0], plt.yticks()[0][-1],5))

plt.title('')
plt.xlabel('')
plt.ylabel('')
plt.savefig('figures/5F_Right.png', dpi=450)